# How to run a model

This Notebook shows how to setup and run a model created in this library with the provided data
and code. 

In [ ]:
import flexible_batch_production as fbp
import pandas as pd
import numpy as np

## Input Data
After importing the required libraries you instatiate the input data depicting the steel plant
setup. Three steel plant setups are given along with this library. 
- Demand Response Steel Making - representing a highly flexible operated green steel plant
- Constant Steel Making - Representing a green steel plant balancing wind generation fluctuation with a large storage system 
- H2-DRI Import - Importing the required DRI sponges from abroad and missing a reduction unit as well as its flexibility potential

By default this example takes the Demand Response Steel Making example. You can change that by changing the corresponding input .dat file

In [ ]:
steel_plant_data = fbp.get_input_dict(f'0_input_data/test_plant_input_data.dat')

Next step is the generation of the wind generation profile for the whole model period. This is generated by given weather data from FINO1 offshore plattform data [1]. 

------

[1] Diese Daten wurden vom Meeresumweltmessnetz des BSH (MARNET), dem RAVE-Projekt (www.rave-offshore.de), dem FINO-Projekt (www.fino-offshore.de) und von Kooperationspartnern des BSH gesammelt und frei zugänglich gemacht.

In [ ]:
generation_data = fbp.generation_data.get_wind_farm_output(
    f'0_input_data/weather/2012_FINO1_processed_weather_data.csv', 
    steel_plant_data['netto_power_wind_park']
)

Last input data are the energy prices for modelling period. As hourly prices are recieved from Agorameter, these have to be converted to the 10 minute resolution of weather data.

In [ ]:
df = pd.read_csv(f'0_input_data/price/2012price_data.csv', sep=';')
prices = df.price.to_list()
price_data = [ten_minutely for hourly in prices for ten_minutely in [hourly]*6]

As the calculation of results for a total year takes extensive amounts of time this example focusses on the optimisation of one week. This is the firs week of the selected 2012 year. As the steel plant is set up to produce 1 million tons of steel in the model period this needs to beadjusted as well.

In [ ]:
start_step = 0
end_step = 1008  # 7 days * 24 hours * (60/10) time steps per hour 
step_size = end_step - start_step

startdate = np.datetime64(f'2012-01-01T00:00') + np.timedelta64(10, 'm') * start_step

steel_plant_data['steel_demand'] = 100

## Model Instantiation and Solving

With all the required input data loaded the model can be instatiated and solved. 
For instantiation an objective needs to be selected. The two available objectives are 
- 'max_profit'
- 'stabilisation'

For stabilitsation the input parameter in the steel_plant_data dictionary 'given_mean_exchange_parameter' needs to be changed to True.

In [ ]:
objective = 'max_profit'  # change to 'stability' for changing objective 
# steel_plant_data['given_goal_load'] = True  # if objective is stability
# steel_plant_data['goal_load'] = 300 # [MW] <- change for changing goal load

fbp_model = fbp.construct.multi_equipment_model(
    steel_plant_data, 
    generation_data[start_step:start_step+step_size],
    price_data[start_step:start_step+step_size],
    objective
)

With gurobi solver available the optimisation should take roughly a minute. If a gurobi license is not available please select another solver. glpk was not tested.

In [ ]:
from pyomo.opt import SolverFactory

print(f"Model with start date {startdate} for {step_size} steps to produce {round(steel_plant_data['steel_demand'], 2)} tons of steel with objective: {objective}\n")

solver = SolverFactory('gurobi')
opt_results = fbp.solve.solve_model(fbp_model,solver=solver, tee=True)

## Analysis
When the model is finished the following methods summarise the model results. 

In [ ]:
figure = fbp.analyse.model_summary_plot(fbp_model, startdate=startdate)

In [ ]:
model_summary = fbp.analyse.summary(fbp_model)

The model results can be stored in a csv file. The analysis of these files is described in [`visualise_results.ipynb`](visualise_results.ipynb).

In [ ]:
fbp.solve.safe_model_results(fbp_model, filename='1_output/model_results.csv')